# Features for DTW

## Import libraries

In [ ]:
# Import module
import pandas as pd
import numpy as np

%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib.path import Path
from matplotlib import image

import os
from preproc import preproc
import pickle

In [ ]:
def show_image(image):

    plt.imshow(image, interpolation='nearest', cmap='gray')
    plt.show()
    
def save_obj(obj, name ):
    with open('obj/'+ name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

## Features functions definitions

In [ ]:
# Returns the index of the pixel composing the lower contour of the shape.
def lowerContour(feature_vector):
#     If there is no black pixels
    if np.size(np.where(feature_vector == 1)) == 0:
        lc = None
    else:
        lc = np.max(np.where(feature_vector == 1))
    return lc


# Returns the index of the pixel composing the upper contour of the shape.
def upperContour(feature_vector):
#     If there is no black pixels
    if np.size(np.where(feature_vector == 1)) == 0:
        uc = None
    else:
        uc = np.min(np.where(feature_vector == 1))
    return uc


# Number of black and white transitions
def transitions(feature_vector):
#     If there is no black pixels
    if np.size(np.where(feature_vector == 1)) == 0:
        return None

    transitions = 0
    for i in range(1,np.size(feature_vector)):
        if feature_vector[i] != feature_vector[i-1]:
            transitions += 1
    
    return transitions
    

# Returns the fraction of blach pixels in the feature vector.
def fracBlackPx(feature_vector):
    nb_bp = (feature_vector == 1).sum()
    fraction = nb_bp/np.size(feature_vector)
    return fraction


# Returns the fraction of blach pixels between the lower and upper contour of the feature vector.
def fracBetweenLcUc(feature_vector, lc, uc):
#     If there is no black pixels    
    if lc is None or uc is None:
        return None

    between = feature_vector[uc:lc+1] # +1 to put in the black pixel of the lower contour
    return fracBlackPx(between)


# Gradient: difference LC[i] , UC[i] to LC[i+1] , UC[i+1]
def gradient(lci, lci1, uci, uci1):
    if uci is None or uci1 is None or lci is None or lci1 is None:
        return None
    if uci1 == lci1:
        return None
    return (uci-lci)/(uci1-lci1)

## Load data

In [ ]:
results, numbers = preproc()
keys = results.keys()

## Creation of feature matices for each images

In [ ]:
# The main dictionary containing all words from all pages.
all_words_features = {}

# For each page
for number in numbers:
    str_number = str(number)
    
#     Pickle files are used to store the dictionary as binaries.
    file = os.path.join('pkl', '%d_features.pkl' % number)

    #If the pickle files don't exists the files are generated by applying the mask and binarisation
    if not os.path.isfile(file):

#         The dictionary of the page containing as keys the id of the words, and as values the matrices of the features.
        page_features_dict = {}

        for key in keys:
#             We work only with words in the current page
            if key[:3] == str_number:

                img = results[key]
                m, n = np.shape(img)
                matrix = []
                prev_lc = -1
                prev_uc = -1

                # each column of the image is a feature vector (1 px wide)
                for i in range(n):
                    fv = img[:,i]
                #     print(f"feature vector {i}: {fv}")

                    lc = lowerContour(fv)
                #     print(f"lower contour: {lc}")

                    uc = upperContour(fv)
                #     print(f"upper contour: {uc}")    

                    t = transitions(fv)
                #     print(f"transitions: {t}")

                    frac_blk_px = fracBlackPx(fv)
                #     print(f"fraction black pixels: {frac_blk_px}")

                    frac_lc_uc = fracBetweenLcUc(fv, lc, uc)
                #     print(f"fraction black pixels lc-uc: {frac_lc_uc}")

                    g = None
                    if prev_lc != -1:
                        g = gradient(prev_lc, lc, prev_uc, uc)

                #     print(f"gradient: {g}")

                    prev_lc = lc
                    prev_uc = uc

                    matrix.append([lc, uc, t, frac_blk_px, frac_lc_uc, g])

#                 Rows are the columns of the image (feature vector) and columns are the different features.
                page_features_dict[key] = matrix


        print('Generating features dictionary for ' + file)
        with open(file, 'wb') as f:
            pickle.dump(page_features_dict, f, pickle.HIGHEST_PROTOCOL)    

        all_words_features.update(page_features_dict)
    
    else:
        #If the files exists, they are just loaded.
        print('Loading feature dictionary from ' + file)
        with open(file, 'rb') as f:
            page_features_dict = pickle.load(f)
            
        all_words_features.update(page_features_dict)
    print('Page %d done' % number)    
print('Loading completed.')